## Hybrid Search
KDB.AI hybrid search is a method of similarity search to increase the relevancy of results retrieved from the vector database. It combines two search methods: sparse vector search, and dense vector search.

Sparse vector search uses the BM25 algorithm to find the most relevant keyword matches, while dense vector search finds the most semantically relevant matches. 

In KDB.AI, users can run sparse or dense search independently, or run hybrid search which runs both sparse and dense vector searches and then re-ranks to combine the results of each search based on a user defined "alpha" value. An alpha value closer to 0 indicates a higher sparse search weight, while a value closer to 1 indicates a higher dense search weight.

In this sample we will use hybrid search over a Federal Reserve speech to extract chunks of the speech that are similar to a user's prompt. In this notebook we will chunk up the document into smaller subsections, create sparse and dense vectors of the chunks, store the vectors in the KDB.AI vector database, and then run dense search, sparse search, and hybrid search to retrieve the most relevant chunks to a user's query. 

Agenda:
1. Imports & Setup
2. Ingest & Chunk Data
3. Generate Sparse & Dense Vectors for Each Chunk
4. Define KDB.AI Session
5. Create KDB.AI Schema & Table
6. Insert Data into KDB.AI Table
7. Create Sparse and Dense Query Vectors
8. Run Sparse, Dense, and Hybrid Searches

[Inflation: Progress and the Path Ahead](https://www.federalreserve.gov/newsevents/speech/powell20230825a.htm)

## 1. Imports & Setup

In [26]:
import pandas as pd
import numpy as np
import os
from getpass import getpass
import kdbai_client as kdbai
import time
from transformers import BertTokenizerFast
from collections import Counter

# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

## 2. Ingest & Chunk Data
Data is from Federal Reserve Chain Jerome H. Powell:

[Inflation: Progress and the Path Ahead](https://www.federalreserve.gov/newsevents/speech/powell20230825a.htm)

In [37]:
### Load the documents we want to prompt an LLM about
doc = TextLoader("data/inflation.txt").load()

In [39]:
### Chunk the documents into 500 character chunks using langchain's text splitter "RucursiveCharacterTextSplitter"
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)

In [40]:
### split_documents produces a list of all the chunks created
pages = [p.page_content for p in text_splitter.split_documents(doc)]

In [41]:
### Create a blank dataframe to store chunks and vectors in before insertion
data = {
    'ID':[],
    'chunk': [],
    'dense': [],
    'sparse': []
}

# Create the DataFrame
df = pd.DataFrame(data)

## 3. Generate Sparse & Dense Vectors for Each Chunk

In [42]:
### Tokenizer to create sparse vectors
token = BertTokenizerFast.from_pretrained('bert-base-uncased')

### Embedding model to be used to embed user input query
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

In [45]:
### Create sparse and dense vectors of each chunk, append to the dataframe

id = 0
for chunk in pages:
    ### Create the dense query vector
    dense_chunk = [embedding_model.encode(chunk).tolist()]

    ### Create the sparse query vector
    sparse_chunk = [dict(Counter(y)) for y in token([chunk], padding=True,max_length=None)['input_ids']]
    sparse_chunk[0].pop(101);sparse_chunk[0].pop(102);

    new_row_df = pd.DataFrame([{"ID": str(id), "chunk": chunk, "dense": dense_chunk[0], "sparse": sparse_chunk[0]}])
    df = pd.concat([df, new_row_df], ignore_index=True)
    id += int(1)
df.head()

,ID,chunk,dense,sparse
0,0,"At last year's Jackson Hole symposium, I delivered a brief, direct message. My remarks this year will be a bit longer, but the message is the same: It is the Fed's job to bring inflation down to our 2 percent goal, and we will do so. We have tightened policy significantly over the past year. Although inflation has moved down from its peak—a welcome development—it remains too high. We are prepared to raise rates further if appropriate, and intend to hold policy at a restrictive level until we","[-0.02285625785589218, -0.02936527691781521, 0.03257254883646965, 0.10052481293678284, 0.04068514332175255, -0.04594222083687782, -0.06294257938861847, -0.012264790944755077, -0.08026789873838425, -0.025917794555425644, -0.04978106915950775, -0.03822582960128784, -0.1009216457605362, 0.045194994658231735, 0.06557615846395493, 0.055128004401922226, 0.00954356323927641, 0.011856192722916603, -0.025397202000021935, -0.005371921695768833, 0.0076469676569104195, -0.004146605730056763, 0.012556673027575016, 0.051397811621427536, -0.018861399963498116, 0.05590061843395233, -0.03667116165161133, -0.0006381056737154722, -0.0489947535097599, -0.00392116280272603, -0.03900369629263878, -0.01048232801258564, -0.012800188735127449, 0.012096517719328403, 0.04901905730366707, 0.04637635499238968, 0.0036269817501306534, -0.006850716192275286, 0.021863503381609917, -0.0403534397482872, -0.003986533731222153, -0.08572252094745636, 0.02243533730506897, -0.05199118331074715, -0.006603453308343887, 0.020408615469932556, 0.020964983850717545, 0.07860825210809708, -0.013116786256432533, 0.03581731766462326, 0.03360438346862793, -0.02600356563925743, 0.037548549473285675, -0.02683015540242195, 0.02880382165312767, 0.06678441166877747, -0.02589700184762478, -0.06794388592243195, -0.00928811077028513, -0.005671375896781683, 0.011809244751930237, -0.0003264880215283483, -0.09315428882837296, 0.028234535828232765, 0.022194689139723778, -0.03869275748729706, 0.006919924635440111, -0.047879934310913086, -0.012619221583008766, 0.059348221868276596, 0.038876794278621674, 0.010870655998587608, 0.00904536247253418, -0.0645502358675003, 0.04553348943591118, 0.0051292432472109795, 0.017876025289297104, 0.06248472258448601, 0.13011716306209564, -0.07174630463123322, 0.09569113701581955, -0.03645782172679901, 0.04733900725841522, -0.09409350156784058, -0.1287020593881607, -0.0757233127951622, 0.015421920455992222, -0.04373693838715553, -0.0005575934192165732, -0.014823147095739841, 0.0886424332857132, -0.00674480851739645, -0.01761506497859955, 0.012838358990848064, 0.019711405038833618, 0.06962540745735168, -0.09802231192588806, -0.029251888394355774, -0.05243441462516785, 0.06309278309345245, ...]","{2012: 2, 2197: 1, 2095: 3, 1005: 2, 1055: 2, 4027: 1, 4920: 1, 17899: 1, 1010: 5, 1045: 1, 5359: 1, 1037: 4, 4766: 1, 3622: 1, 4471: 2, 1012: 4, 2026: 1, 12629: 1, 2023: 1, 2097: 2, 2022: 1, 2978: 1, 2936: 1, 2021: 1, 1996: 4, 2003: 2, 2168: 1, 1024: 1, 2009: 2, 7349: 1, 3105: 1, 2000: 4, 3288: 1, 14200: 2, 2091: 2, 2256: 1, 1016: 1, 3867: 1, 3125: 1, 1998: 2, 2057: 4, 2079: 1, 2061: 1, 2031: 1, 8371: 1, 3343: 2, 6022: 1, 2058: 1, 2627: 1, 2348: 1, 2038: 1, 2333: 1, 2013: 1, 2049: 1, 4672: 1, 1517: 2, 6160: 1, 2458: 1, 3464: 1, 2205: 1, 2152: 1, 2024: 1, 4810: 1, 5333: 1, 6165: 1, 2582: 1, 2065: 1, 6413: 1, 13566: 1, 2907: 1, 25986: 1, 2504: 1, 2127: 1}"
1,1,are confident that inflation is moving sustainably down toward our objective.,"[0.01128313411027193, -0.030178550630807877, -0.028004847466945648, 0.08286452293395996, 0.07045692950487137, -0.012613235972821712, 0.015299422666430473, -0.048042941838502884, -0.014649586752057076, -0.01985168643295765, -0.006772790104150772, -0.012450152076780796, 0.02647615596652031, -0.03598489984869957, -0.03806186839938164, -0.013750843703746796, 0.0006535585271194577, -0.04038465768098831, -0.040352098643779755, 0.08869611471891403, -0.04489782080054283, -0.0018517323769629002, -0.016759781166911125, 0.036626

## 4. Define KDB.AI Session
KDB.AI comes in two offerings:

KDB.AI Cloud - For experimenting with smaller generative AI projects with a vector database in our cloud.
KDB.AI Server - For evaluating large scale generative AI applications on-premises or on your own cloud provider.
Depending on which you use there will be different setup steps and connection details required.

Option 1. KDB.AI Cloud
To use KDB.AI Cloud, you will need two session details - a URL endpoint and an API key. To get these you can sign up for free here.

You can connect to a KDB.AI Cloud session using kdbai.Session and passing the session URL endpoint and API key details from your KDB.AI Cloud portal.

If the environment variables KDBAI_ENDPOINTS and KDBAI_API_KEY exist on your system containing your KDB.AI Cloud portal details, these variables will automatically be used to connect. If these do not exist, it will prompt you to enter your KDB.AI Cloud portal session URL endpoint and API key details.

### Option 1. KDB.AI Cloud

In [ ]:
#Set up KDB.AI endpoing and API key
KDBAI_ENDPOINT = (
    os.environ["KDBAI_ENDPOINT"]
    if "KDBAI_ENDPOINT" in os.environ
    else input("KDB.AI endpoint: ")
)
KDBAI_API_KEY = (
    os.environ["KDBAI_API_KEY"]
    if "KDBAI_API_KEY" in os.environ
    else getpass("KDB.AI API key: ")
)

In [ ]:
### Start Session with KDB.AI Cloud
session = kdbai.Session(api_key=KDBAI_API_KEY, endpoint=KDBAI_ENDPOINT)

### Option 2. KDB.AI Server
To use KDB.AI Server, you will need download and run your own container. To do this, you will first need to sign up for free here.

You will receive an email with the required license file and bearer token needed to download your instance. Follow instructions in the signup email to get your session up and running.

Once the setup steps are complete you can then connect to your KDB.AI Server session using kdbai.Session and passing your local endpoint.

In [46]:
### start session with KDB.AI Server
session = kdbai.Session()

## 5. Create Schema and KDB.AI Table

Now, let us define the schema that will be used to create the KDB.AI table.

"ID" and "chunk" columns will hold the unique identifier and raw text chunk.

sparse and dense columns will hold the respective sparse and dense vectors.

*Note that in the 'sparse' column we define the "b" and "k" parameters. These parameters can be adjusted at runtime, enabling the hyperparameter tuning for term saturation and document length impact on relevance. This will be discussed further during a later example.

In [47]:
schema = dict(
        columns=[
            {"name": "ID", "pytype": "str"},
            {"name": "chunk", "pytype": "str"},
            {
                "name":"sparse",
                "pytype":"dict",
                "sparseIndex":{
                    "k": 1.25,
                    "b": 0.75
                },
            },
            {
                "name":"dense",
                "pytype":"float32",
                "vectorIndex":{
                    "type": "flat",
                    "metric": "L2",
                    "dims": 384
                },
            },
        ]
    )

In [48]:
# If we're re-running this, remove the old trade table
if 'inflation' in session.list():
    table = session.table('inflation')
    table.drop()

In [49]:
table = session.create_table("inflation", schema)

## 6. Insert data into the KDB.AI Table

In [50]:
### Insert the dataframe into the KDB.AI table
table.insert(df)

True

## 7. Create Sparse and Dense Query Vectors

In [51]:
query = '12-month basis'

### Create the dense query vector
dense_query = [embedding_model.encode(query).tolist()]

### Create the sparse query vector
sparse_query = [dict(Counter(y)) for y in token([query], padding=True,max_length=None)['input_ids']]
sparse_query[0].pop(101);sparse_query[0].pop(102);

## 8. Run Sparse, Dense, and Hybrid Searches

In [52]:
### Adjust display settings so we can see full output
pd.set_option('display.max_colwidth', None)

In [53]:
### Type 1 - dense search
table.search(dense_query, n=5)[0][['ID','chunk']]

,ID,chunk
0,9,"coming quarters. Twelve-month core inflation is still elevated, and there is substantial further ground to cover to get back to price stability."
1,35,"That assessment is further complicated by uncertainty about the duration of the lags with which monetary tightening affects economic activity and especially inflation. Since the symposium a year ago, the Committee has raised the policy rate by 300 basis points, including 100 basis points over the past seven months. And we have substantially reduced the size of our securities holdings. The wide range of estimates of these lags suggests that there may be significant further drag in the pipeline."
2,29,"Total hours worked has been flat over the past six months, and the average workweek has declined to the lower end of its pre-pandemic range, reflecting a gradual normalization in labor market conditions (figure 5)."
3,23,"Restrictive monetary policy has tightened financial conditions, supporting the expectation of below-trend growth.5 Since last year's symposium, the two-year real yield is up about 250 basis points, and longer-term real yields are higher as well—by nearly 150 basis points.6 Beyond changes in interest rates, bank lending standards have tightened, and loan growth has slowed sharply.7 Such a tightening of broad financial conditions typically contributes to a slowing in the growth of economic"
4,24,"activity, and there is evidence of that in this cycle as well. For example, growth in industrial production has slowed, and the amount spent on residential investment has declined in each of the past five quarters (figure 4)."


In [54]:
### Type 2 - sparse search
table.search(sparse_query, n=5)[0][['ID','chunk']]

,ID,chunk
0,14,"Similar dynamics are playing out for core goods inflation overall. As they do, the effects of monetary restraint should show through more fully over time. Core goods prices fell the past two months, but on a 12-month basis, core goods inflation remains well above its pre-pandemic level. Sustained progress is needed, and restrictive monetary policy is called for to achieve that progress."
1,8,"On a 12-month basis, core PCE inflation peaked at 5.4 percent in February 2022 and declined gradually to 4.3 percent in July (figure 1, panel B). The lower monthly readings for core inflation in June and July were welcome, but two months of good data are only the beginning of what it will take to build confidence that inflation is moving down sustainably toward our goal. We can't yet know the extent to which these lower readings will continue or where underlying inflation will settle over"
2,6,"On a 12-month basis, U.S. total, or ""headline,"" PCE (personal consumption expenditures) inflation peaked at 7 percent in June 2022 and declined to 3.3 percent as of July, following a trajectory roughly in line with global trends (figure 1, panel A).1 The effects of Russia's war against Ukraine have been a primary driver of the changes in headline inflation around the world since early 2022. Headline inflation is what households and businesses experience most directly, so this decline is very"
3,9,"coming quarters. Twelve-month core inflation is still elevated, and there is substantial further ground to cover to get back to price stability."
4,23,"Restrictive monetary policy has tightened financial conditions, supporting the expectation of below-trend growth.5 Since last year's symposium, the two-year real yield is up about 250 basis points, and longer-term real yields are higher as well—by nearly 150 basis points.6 Beyond changes in interest rates, bank lending standards have tightened, and loan growth has slowed sharply.7 Such a tightening of broad financial conditions typically contributes to a slowing in the growth of economic"


**After comparing the sparse search and dense search results based on the query of "12-month basis", we see that while both return relevant results, the sparse search is returns several chunks that contain specific references to "12-month basis".**

**This search example shows the advantage of having a sparse search when interested in specific terms.** 

Let's run a hybrid search to combine the results:

In [55]:
### Type 3 - hybrid search
table.hybrid_search(dense_vectors=dense_query,sparse_vectors=sparse_query,n=5)[0][['ID','chunk']]

,ID,chunk
0,9,"coming quarters. Twelve-month core inflation is still elevated, and there is substantial further ground to cover to get back to price stability."
1,14,"Similar dynamics are playing out for core goods inflation overall. As they do, the effects of monetary restraint should show through more fully over time. Core goods prices fell the past two months, but on a 12-month basis, core goods inflation remains well above its pre-pandemic level. Sustained progress is needed, and restrictive monetary policy is called for to achieve that progress."
2,23,"Restrictive monetary policy has tightened financial conditions, supporting the expectation of below-trend growth.5 Since last year's symposium, the two-year real yield is up about 250 basis points, and longer-term real yields are higher as well—by nearly 150 basis points.6 Beyond changes in interest rates, bank lending standards have tightened, and loan growth has slowed sharply.7 Such a tightening of broad financial conditions typically contributes to a slowing in the growth of economic"
3,35,"That assessment is further complicated by uncertainty about the duration of the lags with which monetary tightening affects economic activity and especially inflation. Since the symposium a year ago, the Committee has raised the policy rate by 300 basis points, including 100 basis points over the past seven months. And we have substantially reduced the size of our securities holdings. The wide range of estimates of these lags suggests that there may be significant further drag in the pipeline."
4,8,"On a 12-month basis, core PCE inflation peaked at 5.4 percent in February 2022 and declined gradually to 4.3 percent in July (figure 1, panel B). The lower monthly readings for core inflation in June and July were welcome, but two months of good data are only the beginning of what it will take to build confidence that inflation is moving down sustainably toward our goal. We can't yet know the extent to which these lower readings will continue or where underlying inflation will settle over"


##### Hybrid Search with 'alpha = 0.1' Sparse Bias

In [56]:
table.hybrid_search(dense_vectors=dense_query,sparse_vectors=sparse_query,n=5,alpha=0.1)[0][['ID','chunk']]

,ID,chunk
0,14,"Similar dynamics are playing out for core goods inflation overall. As they do, the effects of monetary restraint should show through more fully over time. Core goods prices fell the past two months, but on a 12-month basis, core goods inflation remains well above its pre-pandemic level. Sustained progress is needed, and restrictive monetary policy is called for to achieve that progress."
1,8,"On a 12-month basis, core PCE inflation peaked at 5.4 percent in February 2022 and declined gradually to 4.3 percent in July (figure 1, panel B). The lower monthly readings for core inflation in June and July were welcome, but two months of good data are only the beginning of what it will take to build confidence that inflation is moving down sustainably toward our goal. We can't yet know the extent to which these lower readings will continue or where underlying inflation will settle over"
2,9,"coming quarters. Twelve-month core inflation is still elevated, and there is substantial further ground to cover to get back to price stability."
3,6,"On a 12-month basis, U.S. total, or ""headline,"" PCE (personal consumption expenditures) inflation peaked at 7 percent in June 2022 and declined to 3.3 percent as of July, following a trajectory roughly in line with global trends (figure 1, panel A).1 The effects of Russia's war against Ukraine have been a primary driver of the changes in headline inflation around the world since early 2022. Headline inflation is what households and businesses experience most directly, so this decline is very"
4,23,"Restrictive monetary policy has tightened financial conditions, supporting the expectation of below-trend growth.5 Since last year's symposium, the two-year real yield is up about 250 basis points, and longer-term real yields are higher as well—by nearly 150 basis points.6 Beyond changes in interest rates, bank lending standards have tightened, and loan growth has slowed sharply.7 Such a tightening of broad financial conditions typically contributes to a slowing in the growth of economic"


##### Hybrid Search with 'alpha = 0.9' Dense Bias

In [57]:
table.hybrid_search(dense_vectors=dense_query,sparse_vectors=sparse_query,n=5,alpha=0.9)[0][['ID','chunk']]

,ID,chunk
0,9,"coming quarters. Twelve-month core inflation is still elevated, and there is substantial further ground to cover to get back to price stability."
1,35,"That assessment is further complicated by uncertainty about the duration of the lags with which monetary tightening affects economic activity and especially inflation. Since the symposium a year ago, the Committee has raised the policy rate by 300 basis points, including 100 basis points over the past seven months. And we have substantially reduced the size of our securities holdings. The wide range of estimates of these lags suggests that there may be significant further drag in the pipeline."
2,29,"Total hours worked has been flat over the past six months, and the average workweek has declined to the lower end of its pre-pandemic range, reflecting a gradual normalization in labor market conditions (figure 5)."
3,23,"Restrictive monetary policy has tightened financial conditions, supporting the expectation of below-trend growth.5 Since last year's symposium, the two-year real yield is up about 250 basis points, and longer-term real yields are higher as well—by nearly 150 basis points.6 Beyond changes in interest rates, bank lending standards have tightened, and loan growth has slowed sharply.7 Such a tightening of broad financial conditions typically contributes to a slowing in the growth of economic"
4,24,"activity, and there is evidence of that in this cycle as well. For example, growth in industrial production has slowed, and the amount spent on residential investment has declined in each of the past five quarters (figure 4)."


### Sparse Search Hyperparameter Optimization
#### Dynamic Testing /  Override of b and k using sparse_index_options

Depending on the use-case, it could be beneficial to tune the underlying parameters of sparse search in order to increase relevancy of retrieved data. KDB.AI offers developers the ability to customize the 'b' and 'k' parameters during runtime, ensuring flexibility in sparse search implementation.

**b: (values 0 to 1, defaults to 0.75)** 	
<br>Document length impact on relevance
<br>In general the more specific a document is the less likely length will detrimentally impact relevance so b should be low. For general documents that cover multiple topics at a high level consider using a higher value of b.
<br>
<br>**k: (values 0 to 3, defaults to 1.2)** 
<br>Term saturation
<br>How much more relevant do additional instances of  a term make a document. The lower k, the faster term saturation occurs, (i.e. additional terms do not count as much).

In [ ]:
table.hybrid_search(dense_vectors=dense_query,sparse_vectors=sparse_query,n=5,sparse_index_options={'b':0.1, 'k':3})[0][['ID','chunk']]

**Additionally, upon the insertion of new data into the KDB.AI table, all underlying BM25 statistics are updated. This means that when new data is added, the BM25 scoring is updated and aligns with the all sparse data when a sparse seach is run.**

### Delete the KDB.AI Table
Once finished with the table, it is best practice to drop it.

In [16]:
table.drop()

True